In [1]:
# 2025/7/13
# zhangzhong
# TODO: 超大数据集无法全部放到内存中的问题，目前比较适合我的解决方法是keepinmemory = False, 或者看看streaming=True能不能和多核一起工作，尝试一下这两种方案


In [5]:
from datasets import load_dataset_builder
from pprint import pprint
from datasets import load_dataset

# c4_builder = load_dataset_builder("allenai/c4", "en")
# pprint(c4_builder.info)

In [9]:
import multiprocessing

num_proc = multiprocessing.cpu_count() // 2
print(f"Number of processes: {num_proc}")

Number of processes: 24


In [4]:
from datasets import load_dataset

# https://huggingface.co/datasets/allenai/c4
# 加载的时候使用多核根本没用和                     
original_c4 = load_dataset(path="allenai/c4", name="en", split="train[:10000]")
print(original_c4)

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Dataset({
    features: ['text', 'timestamp', 'url'],
    num_rows: 10000
})


In [5]:
# 卧槽，这个数据集太大，只是加载就需要十几分钟
original_c4['text'][:10], original_c4['timestamp'][:10], original_c4['url'][:10]

(['Beginners BBQ Class Taking Place in Missoula!\nDo you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now. Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers. He will be teaching a beginner level class for everyone who wants to get better with their culinary skills.\nHe will teach you everything you need to know to compete in a KCBS BBQ competition, including techniques, recipes, timelines, meat selection and trimming, plus smoker and fire information.\nThe cost to be in the class is $35 per person, and for spectators it is free. Included in the cost will be either a t-shirt or apron and you will be tasting samples of each meat that is prepared.',
  'Discussion in \'Mac OS X Lion (10.7)\' started by axboi87, Jan 20, 2012.\nI\'ve got a 500gb internal drive and a 240gb SSD.\nWhen trying to restore using disk utility i\'m given the error "Not enough space on disk ____ to restore"\nBut I sho

In [6]:
# 定义一个多核文本预处理函数
def preprocess(example):
    text = example['text'].strip()  # 去除首尾空格
    return {'text': text}

In [7]:
# 使用dataset.map方法进行多核预处理
preprocessed_c4 = original_c4.map(
    preprocess,
    remove_columns=original_c4.column_names, # 去掉数据集中原本的列，节省内存
    num_proc=num_proc,  # 使用多核处理, 这个确实有用
    desc="Preprocessing C4 dataset"
)

In [8]:
print(preprocessed_c4)

# 我们应该是需要把所有的数据集都处理成只有text一列的形式
# 然后再concat起来，就可以用来训练bpe了


Dataset({
    features: ['text'],
    num_rows: 10000
})


In [9]:
# wiki_builder = load_dataset_builder(path="wikimedia/wikipedia", name="20231101.en")
# pprint(wiki_builder.info)

In [10]:
# # https://huggingface.co/datasets/wikimedia/wikipedia
# split可以直接在huggingface对应的dataset页面上查看
original_wiki = load_dataset(path="wikimedia/wikipedia", name="20231101.en", split="train[:10000]")
print(original_wiki)

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 10000
})


In [11]:
def preprocess_wiki(example):
    text = example['text'].strip()  # 去除首尾空格
    return {'text': text}

In [12]:
preprocessed_wiki = original_wiki.map(
    preprocess_wiki,
    remove_columns=original_wiki.column_names,  # 去掉数据集中原本的列，节省内存
    num_proc=num_proc,  # 使用多核处理
    desc="Preprocessing Wikipedia dataset"
)

In [13]:
print(preprocessed_wiki)

Dataset({
    features: ['text'],
    num_rows: 10000
})


In [3]:
# 3G
# https://huggingface.co/datasets/nampdn-ai/tiny-codes

tiny_codes_builder = load_dataset_builder("nampdn-ai/tiny-codes")
pprint(tiny_codes_builder.info)




Using the latest cached version of the dataset since nampdn-ai/tiny-codes couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /data2/huggingface/datasets/nampdn-ai___tiny-codes/default/0.0.0/9aebe5ee8b406356d5f5f2d603bc0a1684ee8ce7 (last modified on Sun Jul 13 19:12:32 2025).


DatasetInfo(description='',
            citation='',
            homepage='',
            license='',
            features={'action_verb': Value('large_string'),
                      'adjective': Value('large_string'),
                      'common_sense_topic': Value('large_string'),
                      'idx': Value('int64'),
                      'main_topic': Value('large_string'),
                      'programming_language': Value('large_string'),
                      'prompt': Value('large_string'),
                      'response': Value('large_string'),
                      'scenario': Value('large_string'),
                      'subtopic': Value('large_string'),
                      'target_audience': Value('large_string')},
            post_processed=None,
            supervised_keys=None,
            builder_name='parquet',
            dataset_name='tiny-codes',
            config_name='default',
            version=0.0.0,
            splits={'train': SplitInfo(name='

In [6]:
tiny_codes = load_dataset(path="nampdn-ai/tiny-codes", split="train[:10000]")
print(tiny_codes)

Dataset({
    features: ['prompt', 'main_topic', 'subtopic', 'adjective', 'action_verb', 'scenario', 'target_audience', 'programming_language', 'common_sense_topic', 'idx', 'response'],
    num_rows: 10000
})


In [12]:
# 因为只是在train tokenizer 所以我就把 prompt和response 用空格 拼接起来就好了
def preprocess_tiny_codes(example):
    text = f"{example['prompt']} {example['response']}".strip()  # 去除首尾空格
    return {'text': text}
preprocessed_tiny_codes = tiny_codes.map(
    preprocess_tiny_codes,
    remove_columns=tiny_codes.column_names,  # 去掉数据集中原本的列
    num_proc=num_proc,  # 使用多核处理
    desc="Preprocessing Tiny Codes dataset"
)
print(preprocessed_tiny_codes[:10])

{'text': ['Develop a Python program snippet to Determine High Sneezing and coughing etiquette: Preventing Spread of Germs for Engineer for Experts. Incorporate if/else or switch/case statements to handle various cases related to the Bias. Dry-run, ensure your control flow logic is clear and well-commented. Here is python code which determines high sneezing and coughing etiquette based upon different conditions :\n\n\n```python \ndef determine_high_sneeze_cough_etiquette(isolation_period):\n    # check whether isolation period has been exceeded \n    if (isolation_period >0) & (today() - last_symptomatic_day() < isolation_period):\n        return "Stay home"\n    \n    elif exposure_risk() >= risk_threshold():\n        return "Wear mask"\n        \n    else: \n        return "Go about usual activities"\n    \n        \n# define function to calculate days since symptoms appeared    \ndef today():\n    return datetime.now().date()\n    \n    \n# define function to retrieve date when patie

In [13]:
# https://huggingface.co/datasets/manu/project_gutenberg/

gutenberg_builder = load_dataset_builder("manu/project_gutenberg")
pprint(gutenberg_builder.info)

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

DatasetInfo(description='',
            citation='',
            homepage='',
            license='',
            features={'id': Value('string'), 'text': Value('string')},
            post_processed=None,
            supervised_keys=None,
            builder_name='parquet',
            dataset_name='project_gutenberg',
            config_name='default',
            version=0.0.0,
            splits={'de': SplitInfo(name='de',
                                    num_bytes=1070196924,
                                    num_examples=3131,
                                    shard_lengths=[2044, 1087],
                                    dataset_name='project_gutenberg'),
                    'en': SplitInfo(name='en',
                                    num_bytes=25616345280,
                                    num_examples=61340,
                                    shard_lengths=[1180,
                                                   1180,
                                             

In [38]:
gutenberg = load_dataset(path="manu/project_gutenberg", split="en[:10000]")
print(gutenberg)

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'text'],
    num_rows: 10000
})


In [39]:
pprint(gutenberg[:2])

{'id': ['41496-8', '41496-8'],
 'text': ['The Project Gutenberg eBook, Addison, by William John Courthope\n'
          '\n'
          '\n'
          'This eBook is for the use of anyone anywhere at no cost and with\n'
          'almost no restrictions whatsoever.  You may copy it, give it away '
          'or\n'
          're-use it under the terms of the Project Gutenberg License '
          'included\n'
          'with this eBook or online at www.gutenberg.org\n'
          '\n'
          '\n'
          '\n'
          '\n'
          '\n'
          'Title: Addison\n'
          '\n'
          '\n'
          'Author: William John Courthope\n'
          '\n'
          '\n'
          '\n'
          'Release Date: November 27, 2012  [eBook #41496]\n'
          '\n'
          'Language: English\n'
          '\n'
          'Character set encoding: ISO-8859-1\n'
          '\n'
          '\n'
          '***START OF THE PROJECT GUTENBERG EBOOK ADDISON***\n'
          '\n'
          '\n'
        

In [45]:
import re

def strip_gutenberg_headers(text: str) -> str:
    """
    使用 Project Gutenberg 通用标记提取正文内容。
    """
    start_re = re.compile(r"\*\*\*\s*START OF (THIS|THE) PROJECT GUTENBERG EBOOK.*?\*\*\*", re.IGNORECASE)
    end_re = re.compile(r"\*\*\*\s*END OF (THIS|THE) PROJECT GUTENBERG EBOOK.*?\*\*\*", re.IGNORECASE)

    start_match = start_re.search(text)
    end_match = end_re.search(text)

    if not start_match or not end_match:
        print("⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。")
        return text.strip()

    body = text[start_match.end():end_match.start()]
    return body.strip()


def preprocess_gutenberg(example):
    # 去掉开头的header，版权信息，等等内容
    # 算了，麻烦。。。
    text = strip_gutenberg_headers( example['text'])
    return {'text': text}



In [46]:
processed_gutenberg = gutenberg.map(
    preprocess_gutenberg,
    remove_columns=gutenberg.column_names,  # 去掉数据集中原本的列
    num_proc=num_proc,  # 使用多核处理
    desc="Preprocessing Gutenberg dataset"
)

Preprocessing Gutenberg dataset (num_proc=24):   0%|          | 0/10000 [00:00<?, ? examples/s]

⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。
⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。

⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。
⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。
⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。
⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。
⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。
⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。

⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。
⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。
⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。
⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。

⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。
⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。

⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。

⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。
⚠️ 无法找到标准 Project Gutenberg 标记，可能是非标准格式。

⚠️ 无法找到标准 Project

In [41]:
pprint(processed_gutenberg[0])

{'text': 'PAGE\n'
         '\n'
         '  CHAPTER I.\n'
         '    THE STATE OF ENGLISH SOCIETY AND LETTERS\n'
         '      AFTER THE RESTORATION                        1\n'
         '\n'
         '  CHAPTER II.\n'
         "    ADDISON'S FAMILY AND EDUCATION                21\n"
         '\n'
         '  CHAPTER III.\n'
         '    ADDISON ON HIS TRAVELS                        38\n'
         '\n'
         '  CHAPTER IV.\n'
         '    HIS EMPLOYMENT IN AFFAIRS OF STATE            53\n'
         '\n'
         '  CHAPTER V.\n'
         '    THE "TATLER" AND "SPECTATOR"                  78\n'
         '\n'
         '  CHAPTER VI.\n'
         '    "CATO"                                       110\n'
         '\n'
         '  CHAPTER VII.\n'
         "    ADDISON'S QUARREL WITH POPE                  125\n"
         '\n'
         '  CHAPTER VIII.\n'
         '    THE LAST YEARS OF HIS LIFE                   139\n'
         '\n'
         '  CHAPTER IX.\n'
         '    THE GENIUS 

In [48]:
# https://huggingface.co/datasets/eminorhan/gutenberg_en
# 找到一个已经处理好的gutenberg数据集
# 太好了！

gutenberg_en_builder = load_dataset_builder(path="eminorhan/gutenberg_en", name="chunk_size_2048")
pprint(gutenberg_en_builder.info)

DatasetInfo(description='',
            citation='',
            homepage='',
            license='',
            features=None,
            post_processed=None,
            supervised_keys=None,
            builder_name='json',
            dataset_name='gutenberg_en',
            config_name='chunk_size_2048',
            version=0.0.0,
            splits=None,
            download_checksums=None,
            download_size=None,
            post_processing_size=None,
            dataset_size=None,
            size_in_bytes=None)


In [49]:
gutenberg_en = load_dataset(path="eminorhan/gutenberg_en", name="chunk_size_2048")

gutenberg_en_paragraph_2048.jsonl:   0%|          | 0.00/24.2G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
def preprocess_gutenberg_en(example):
    # 这个数据集已经处理好了，所以直接返回text就行了
    return {'text': example['text']}

In [23]:
# 14G
# https://huggingface.co/datasets/donfu/oa-stackexchange
stack_exchange_builder = load_dataset_builder("donfu/oa-stackexchange")
pprint(stack_exchange_builder.info)


DatasetInfo(description='',
            citation='',
            homepage='',
            license='',
            features={'INSTRUCTION': Value('string'),
                      'METADATA': {'answer_score': Value('int64'),
                                   'question_score': Value('int64'),
                                   'tags': Value('string')},
                      'RESPONSE': Value('string'),
                      'SOURCE': Value('string')},
            post_processed=None,
            supervised_keys=None,
            builder_name='parquet',
            dataset_name='oa-stackexchange',
            config_name='default',
            version=0.0.0,
            splits={'train': SplitInfo(name='train',
                                       num_bytes=6551421448,
                                       num_examples=6331083,
                                       shard_lengths=[488221,
                                                      472221,
                                     

In [24]:
stack_exchange = load_dataset(path="donfu/oa-stackexchange", split="train[:1%]")

In [25]:
print(stack_exchange)

Dataset({
    features: ['INSTRUCTION', 'RESPONSE', 'SOURCE', 'METADATA'],
    num_rows: 63311
})


In [26]:
def preprocess_stack_exchange(example):
    # 只保留question和answer
    text = f"{example['INSTRUCTION']} {example['RESPONSE']}".strip()  # 去除首尾空格
    return {'text': text}

preprocessed_stack_exchange = stack_exchange.map(
    preprocess_stack_exchange,
    remove_columns=stack_exchange.column_names,  # 去掉数据集中原本的列
    num_proc=num_proc,  # 使用多核处理
    desc="Preprocessing Stack Exchange dataset"
)
print(preprocessed_stack_exchange[0])

Preprocessing Stack Exchange dataset (num_proc=24):   0%|          | 0/63311 [00:00<?, ? examples/s]

{'text': "Did the Greeks build temples for all of the children of Cronus?\nI know that the Ancient Greeks built temples for a lot of the children of Cronus, such as the Temple of Zeus and the Temple of Hera. Were temples built for all of the children of Cronus, or were they only built for the major ones? If some were left out, is there any indication of why? According to the traditional geneaology of the Greek Gods, Cronus had six children with Rhea: Hestia, Zeus, Hera, Poseidon, Hades and Demeter. All children were part of at least one major variation of the Twelve Olympians and - with the exception of Hestia - had temples honouring them. \n\nHestia was Cronus firstborn and her sanctuary was the hearth of the Prytaneion, the political and religious center of the community. Also, Hestia was thought of as being present in every offering fire in all temples and received first offering in sacrifices. Thus, she didn't need temples of her own.\n\nOf course there may have been temples honour

In [14]:
from datasets import concatenate_datasets

combined_dataset = concatenate_datasets([preprocessed_c4, preprocessed_wiki])

In [15]:
print(combined_dataset)

Dataset({
    features: ['text'],
    num_rows: 20000
})


In [16]:

# 咱们把训练好的文本写到一个txt文件里面吧
# 或许从txt文件里面加载会快一些
with open("corpus.txt", "w", encoding="utf-8") as f:
    for text in combined_dataset['text']:
        f.write(text + "\n")


In [ ]:
# tokenizer

from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers import normalizers

tokenizer = Tokenizer(models.BPE())
tokenizer.normalizer = None  # No normalization, default for GPT2
# tokenizer.normalizer = normalizers.Sequence([
#     normalizers.NFKC(),  
# ])
tokenizer.pre_tokenizer = pre_tokenizers.Sequence([

# 1. add_prefix_space: bool = True

# ✅ 功能解释：
# 	•	是否在文本开头添加一个空格（" "），仅在文本不是以空格开头时；
# 	•	这个空格会被当成 GPT-style 的特殊 Ġ 前缀（Unicode U+0120）；
# 	•	有助于模型区分词的开头 vs 词中间的空格。


# 2. use_regex: bool = False

# ✅ 功能解释：
# 	•	是否使用正则表达式来切分文本中的 token；
# 	•	默认为 False，意味着采用逐字节（byte-level）方式处理，不会使用正则切分单词。
    

# 综合了chatgpt和deepseek的答案
# 目前看来，采用ByteLevel的默认设置，也就是两个参数都设置为True是比较合理的
# https://chat.deepseek.com/a/chat/s/ca987739-6346-4dec-ae7b-36835c2b6541

pre_tokenizers.ByteLevel(add_prefix_space=True, use_regex=True)
]) 
tokenizer.decoder = decoders.Sequence([
decoders.ByteLevel()
]) 

trainer = trainers.BpeTrainer(
    vocab_size=3200,
    # min_frequency=2,
    special_tokens=[ "</s>"],
    show_progress=True,
    initial_alphabet=ByteLevel.alphabet()  # ← key point: includes all 256 bytes
)

In [ ]:
# 卧槽，直接从文件里读真的快！三分钟竟然就train完了！
# 哦了，那我懂了呀，咱们就直接把不同的数据集都采样一些，然后组织成多个文件
# 总共就128G就可以了，不需要太大，毕竟只是一个tokenizer
# 因为我们的机器的内存是256G的 是可以全部加载进来的，应该会快一些
# 然后这个bpe的训练算法是串行的，所以多核也是没用的，反而单核高频会好一些。
tokenizer.train(files=["corpus.txt"], trainer=trainer)

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm


def batch_iterator():
    for example in tqdm(combined_dataset):
        yield example['text']

# 感觉这样也不是很快啊
# 咱们先把所有的预处理结束之后的东西放到一个txt文件中
# 然后在读文件来训练吧，看看快不快
# 咱们再把数据集的规模缩小一点


# def make_batch_iterator(dataset, max_workers=num_proc, buffer_size=10000):
#     def get_text(example):
#         return example['text'].strip()

#     def iterator():
#         with ThreadPoolExecutor(max_workers=max_workers) as executor:
#             # 将样本分批异步处理（启动任务）
#             futures = [executor.submit(get_text, ex) for ex in dataset]

#             # 异步消费
#             for future in tqdm(as_completed(futures), total=len(dataset), desc="Feeding tokenizer"):
#                 text = future.result()
#                 if text and len(text) > 30:
#                     yield text

#     return iterator()

# 5分钟没有train完
# tokenizer.train_from_iterator(
#     # make_batch_iterator(combined_dataset),
#     iterator=batch_iterator(),
#     trainer=trainer,
#     length=len(combined_dataset)  # 这个参数是可选的，表示数据集的长度
# )

100%|██████████| 20000/20000 [00:05<00:00, 3569.05it/s]


: 

: 

In [ ]:
tokenizer.save("byte_level_bpe_tokenizer.json")

In [ ]:
# 咱们测试一下这个tokenizer吧
def test_tokenizer(sentence: str) -> None:
    tokens = tokenizer.encode(sentence)
    print(tokens.tokens)  # Output the tokens
    print(tokens.ids)       # Output the token IDs
    # how to do decode
    decoded = tokenizer.decode(tokens.ids)
    print(decoded)  # Output the decoded string

In [ ]:
test_tokenizer("Hello, world! This is a test sentence for the tokenizer.")

[]
[]



In [ ]:
# 好像这个训练出来的tokenizer会在句子的开头加上一个 空格 作为prefix
# 这个应该是有参数可以控制的
# pre_tokenizers.ByteLevel(add_prefix_space=True,use_regex=False)
test_tokenizer("the")

[]
[]



In [ ]:
test_tokenizer("Byte-level BPE supports emojis 👋 and symbols ©®👍.")

[]
[]

